# Problem 4 Solution

I chose to do problem 4 from the list of screener questions. I didn't look at the qiskit implementation, except to compare the output of my code and that of qiskit.

## Qiskit Circuit
To validate my implementation, first we create a simple circuit in qiskit and run simluation on it.

In [24]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(5, 5)
qc.h(0)
qc.h(1)
qc.cz(0, 4)
qc.unitary([[0.5 + 0.5j, 0.5 - 0.5j], [0.5 - 0.5j, 0.5 + 0.5j]], 1)
out = qc.measure([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])
qc.draw()


┌───┐              ┌─┐      
q_0: ┤ H ├────────────■─┤M├──────
     ├───┤┌─────────┐ │ └╥┘┌─┐   
q_1: ┤ H ├┤ unitary ├─┼──╫─┤M├───
     └┬─┬┘└─────────┘ │  ║ └╥┘   
q_2: ─┤M├─────────────┼──╫──╫────
      └╥┘     ┌─┐     │  ║  ║    
q_3: ──╫──────┤M├─────┼──╫──╫────
       ║      └╥┘     │  ║  ║ ┌─┐
q_4: ──╫───────╫──────■──╫──╫─┤M├
       ║       ║         ║  ║ └╥┘
c: 5/══╩═══════╩═════════╩══╩══╩═
       2       3         0  1  4

In [25]:
from qiskit import Aer, execute

def simulate(circuit, num_shots):
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(circuit, backend_sim, shots=num_shots)
    result_sim = job_sim.result()
    return result_sim.get_counts(circuit)

qiskit_counts = simulate(qc, num_shots=10000)

## JQiskit Circuit
Then I built the same circuit in my simulator and ran it.

In [26]:
from jqiskit.api import QuantumCircuit as JQuantumCircuit

qc = JQuantumCircuit(5)
qc.h(0)
qc.h(1)
qc.cz(0, 4)
qc.sqrtnot(1)
print(qc)
jqiskit_counts, state = qc.measure(num_shots=10000)

Hadamard: (0,)
Hadamard: (1,)
CZ: (0, 4)
SQRTNOT: (1,)


## Validation

I then compared the two by looking at the simulation output states. Note how qiskit is the opposite endian-ness as my implementation, so I had to flip the state to do a proper comparison.

In [27]:
assert len(jqiskit_counts) == len(qiskit_counts), "Number of states don't match!"
print(len(qiskit_counts))

for state_str in jqiskit_counts:
    print(f'state: {state_str}, qiskit: {qiskit_counts[state_str[::-1]]}; jqiskit: {jqiskit_counts[state_str]}')

4
state: 01000, qiskit: 2516; jqiskit: 2576
state: 11000, qiskit: 2424; jqiskit: 2476
state: 10000, qiskit: 2526; jqiskit: 2491
state: 00000, qiskit: 2534; jqiskit: 2457


In [28]:
import numpy as np

qc = JQuantumCircuit(2)
qc.parametric('[[1, 0.], [0.0 , exp(1.0j*theta)]]', 0)
theta = np.pi / 7
qc.measure(feed_dict={'theta': theta})

1


({'00': 1000}, array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]))

In [29]:
qc = QuantumCircuit(2, 2)
qc.unitary([[1, 0.], [0.0, np.exp(1.0j*theta)]], 0)
qc.measure([0, 1], [0, 1])
simulate(qc, 10000)

{'00': 10000}

In [30]:
import numpy as np
from typing import List

def generate_swaps(targets: List[int]):
    
    swap_list = []
    min_target = min(targets)
    max_target = max(targets)
    
    offset = max_target - min_target
    tmp = np.full((max_target - min_target + 1, ), np.nan)
    
    for idx, target in enumerate(targets):
        tmp[target - min_target] = idx
    for idx in range(len(targets)):
        tmp_idx = np.where(tmp == idx)[0][0]
        for jdx in reversed(range(idx + 1, tmp_idx + 1)):
            swap_list.append((jdx - 1 + min_target, jdx + min_target))
            tmp[jdx], tmp[jdx - 1] =  tmp[jdx - 1], tmp[jdx]
            
    return swap_list
        
print(generate_swaps([9, 3, 5]))
print(generate_swaps([0]))
print(generate_swaps([0, 1]))
print(generate_swaps([1, 0]))
print(generate_swaps([3, 6]))
print(generate_swaps([5, 3]))

[(8, 9), (7, 8), (6, 7), (5, 6), (4, 5), (3, 4), (5, 6)]
[]
[]
[(0, 1)]
[(5, 6), (4, 5)]
[(4, 5), (3, 4)]


In [31]:
foo = [] + [1,2] + [3] + list(reversed([1,2]))

In [32]:
foo += [4]

In [10]:
foo

[1, 2, 3, 2, 1, 4]